결막염 conjunctivitis  
궤양각막 corneal_ulcer  
각막 corneal  
백내장 Cataract  
색소침착성 각막염 PIH  

In [1]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import cv2
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.cm as cm

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import pickle
from tqdm.notebook import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# datagen = ImageDataGenerator(
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
# )

In [5]:
# !gdown --id /content/drive/MyDrive/image_recognition/data/image_data/img/Cataract.zip

In [6]:
# !gdown --/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract.zip

In [7]:
# 파일 압축 해제
import zipfile

with zipfile.ZipFile('/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/image_recognition/data/image_data/img/')

In [8]:
folder_path = '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract'
label_names = os.listdir(folder_path)
label_names 

['m', 'u']

In [9]:
# import shutil

# try:
#     shutil.rmtree('/content/resize')
# except FileNotFoundError:
#     pass

# !mkdir /content/resize

In [10]:
os.mkdir('/content/resize')
for label in label_names:     
    dir_path = '/content/resize/'+ label
    os.mkdir(dir_path)

In [11]:
# 카테고리별로 이미지파일 경로 리스트 사전 만들기
# keys : 카테고리명
# values : 리스트 of 이미지 경로들
dataset = {} # 카테고리명 : [이미지1경로, 이미지2경로, ...]

for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'m': ['/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_03fa5209-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_040ccfb5-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_0421a31d-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_0448a057-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_05188a95-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_054726ca-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_05568c4e-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_05c13e63-60a5-11ec-8402-0a7404

## Resize with padding

In [12]:
label2index = {'m' : 0, 'u' : 1}

In [13]:
def resize_img(img_path, img_size=224):    
    img = cv2.imread(img_path)

    if(img.shape[1] > img.shape[0]) : 
        ratio = img_size/img.shape[1]
    else :
        ratio = img_size/img.shape[0]

    img = cv2.resize(img, dsize=(0, 0), fx=ratio, fy=ratio, interpolation=cv2.INTER_LINEAR)

    # 그림 주변에 검은색으로 칠하기
    w, h = img.shape[1], img.shape[0]

    dw = (img_size-w)/2 # img_size와 w의 차이
    dh = (img_size-h)/2 # img_size와 h의 차이

    M = np.float32([[1,0,dw], [0,1,dh]])  #(2*3 이차원 행렬)
    img_re = cv2.warpAffine(img, M, (224, 224)) #이동변환  
    cv2.imwrite('/content/resize/{0}/{1}'.format(label, img_path.split("/")[-1]) , img_re)

In [14]:
# 이미지 리사이즈
# 1. 이미지 하나씩 읽어들여서
# 2. 리사이즈한다음
# 3. images, labels 생성
for label, img_paths in dataset.items():
    for img_path in img_paths:
        resize_img(img_path, img_size=224)

In [40]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
import splitfolders
splitfolders.ratio('/content/resize', output='dataset', seed=77, ratio=(0.6, 0.2, 0.2))

Copying files: 13660 files [00:02, 5580.11 files/s]


In [16]:
folder_path = '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'m': ['/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_03fa5209-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_040ccfb5-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_0421a31d-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_0448a057-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_05188a95-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_054726ca-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_05568c4e-60a5-11ec-8402-0a7404972c70.png',
  '/content/drive/MyDrive/image_recognition/data/image_data/img/Cataract/m/crop_D0_05c13e63-60a5-11ec-8402-0a7404

In [17]:
x_train = []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename) # img를 array 형태로 변경

        x_train.append(image)

x_train = np.array(x_train, dtype=object)

In [42]:
x_train.shape

(13660, 224, 224, 3)

In [37]:
n_train = x_train.shape[0]

In [20]:
# n_train

13660

In [21]:
# zero-centering
compute_mean = lambda imgs : np.mean(imgs, axis=0)

In [23]:
mean_img = compute_mean(x_train)
mean_img.shape

(224, 224, 3)

In [24]:
sub_mean_img = lambda image : image.astype('int8') - mean_img.astype('int8')

## ImageDataGenerator 설정

In [66]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=sub_mean_img
)

val_datagen = ImageDataGenerator(
    preprocessing_function=sub_mean_img)

test_datagen = ImageDataGenerator(
    preprocessing_function=sub_mean_img)

#### flow_from_directory 이용하기

In [67]:
batch_size = 32

In [68]:
train_generator = train_datagen.flow_from_directory('/content/dataset/train',batch_size=batch_size, target_size=(224, 224))
val_generator = val_datagen.flow_from_directory('/content/dataset/val',batch_size=batch_size, target_size=(224, 224))
test_generator = test_datagen.flow_from_directory('/content/dataset/test',batch_size=batch_size, target_size=(224, 224))

Found 8195 images belonging to 2 classes.
Found 2731 images belonging to 2 classes.
Found 2734 images belonging to 2 classes.


## 모델 학습 및 검증

In [69]:
# modeling
def create_model():
    model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(224, 224, 3)),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(2)
            ])
    
    return model

In [70]:
model = create_model()

model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [71]:
# 한 epoch 수 / batch size = 32 기준 
steps_per_epoch = n_train // batch_size
steps_per_epoch

426

In [72]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, 
          validation_data=val_generator, epochs=3)

Epoch 1/3
257/426 [=================>............] - ETA: 2:10 - loss: 1.7670 - accuracy: 0.6935

426/426 [==============================] - 222s 518ms/step - loss: 1.7670 - accuracy: 0.6935 - val_loss: 1.5738 - val_accuracy: 0.6595
